In [21]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from helpers import recursive_find, read_json, create_fedora_results_table

In [22]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("artifacts/results/fedora", "*.json"))
print('Found %s experiment library files.' % len(experiments))

Found 10758 experiment library files.


In [23]:
df = create_fedora_results_table(experiments)
display(df)

,a,b,original,changed,analysis,seconds,predictor,prediction
0,fedora-libs-35,fedora-libs-37,first/libQt5Quick.so.5.15.5,second/libQt5Quick.so.5.15.5,missing-previously-found-symbols,0.002813,symbols,True
1,fedora-libs-35,fedora-libs-37,first/libQt5Quick.so.5.15.5,second/libQt5Quick.so.5.15.5,missing-previously-found-exports,0.254318,symbols,True
2,fedora-libs-35,fedora-libs-37,first/libQt5Quick.so.5.15.5,second/libQt5Quick.so.5.15.5,abidiff,0.0563,libabigail,True
3,fedora-libs-35,fedora-libs-37,first/libQt5Quick.so.5.15.5,second/libQt5Quick.so.5.15.5,abi-compliance-tester,0.181996,abi-laboratory,False
4,fedora-libs-34,fedora-libs-37,first/libx11globalcomm.so.1.0.0,second/libx11globalcomm.so.1.0.0,missing-previously-found-symbols,0.000051,symbols,True
...,...,...,...,...,...,...,...,...
43027,fedora-libs-34,fedora-libs-36,first/kimg_dds.so,second/kimg_dds.so,abi-compliance-tester,0.08652,abi-laboratory,False
43028,fedora-libs-34,fedora-libs-35,first/libkdeinit_kcmshell.so,second/libkdeinit_kcmshell.so,missing-previously-found-symbols,0.00011,symbols,True
43029,fedora-libs-34,fedora-libs-35,first/libkdeinit_kcmshell.so,second/libkdeinit_kcmshell.so,missing-previously-found-exports,0.000028,symbols,True
43030,fedora-libs-34,fedora-libs-35,first/libkdeinit_kcmshell.so,second/libkdeinit_kcmshell.so,abidiff,0.01076,libabigail,True


In [15]:
# disagree = df.groupby(["a","b", "original", "changed"])['prediction'].nunique() > 1
# disagree = pandas.DataFrame(disagree)
# Where all predictors agree
all_predictors_agree = pandas.DataFrame(df.groupby(['a', 'b', 'original', 'changed'])['prediction'].nunique() == 1).reset_index()
df_all_agree = df.merge(all_predictors_agree, on=['a', 'b', 'original', 'changed'], suffixes=["", "_all_agree"])
df_all_agree = df_all_agree[df_all_agree['prediction_all_agree']]
df_all_agree

,a,b,original,changed,analysis,seconds,predictor,prediction,prediction_all_agree
2208,fedora-libs-34,fedora-libs-35,first/libgdal.so,second/libgdal.so.30.0.3,missing-previously-found-symbols,0.003849,symbols,False,True
2209,fedora-libs-34,fedora-libs-35,first/libgdal.so,second/libgdal.so.30.0.3,missing-previously-found-exports,0.041891,symbols,False,True
2210,fedora-libs-34,fedora-libs-35,first/libgdal.so,second/libgdal.so.30.0.3,abidiff,49.790041,libabigail,False,True
2211,fedora-libs-34,fedora-libs-35,first/libgdal.so,second/libgdal.so.30.0.3,abi-compliance-tester,0.111954,abi-laboratory,False,True
6376,fedora-libs-34,fedora-libs-37,first/mmap.so,second/mmap.cpython-310-x86_64-linux-gnu.so,missing-previously-found-symbols,0.000021,symbols,False,True
...,...,...,...,...,...,...,...,...,...
41371,fedora-libs-35,fedora-libs-37,first/syslog.cpython-310-x86_64-linux-gnu.so,second/syslog.so,abi-compliance-tester,0.09968,abi-laboratory,False,True
41740,fedora-libs-35,fedora-libs-37,first/resource.so,second/resource.cpython-310-x86_64-linux-gnu.so,missing-previously-found-symbols,0.000015,symbols,False,True
41741,fedora-libs-35,fedora-libs-37,first/resource.so,second/resource.cpython-310-x86_64-linux-gnu.so,missing-previously-found-exports,0.000005,symbols,False,True
41742,fedora-libs-35,fedora-libs-37,first/resource.so,second/resource.cpython-310-x86_64-linux-gnu.so,abidiff,0.008399,libabigail,False,True


In [16]:
# Convert to wide format where analysis and predictor are columns.
df_wide = pandas.pivot(df, columns=['analysis', 'predictor'], index=['a', 'b', 'original', 'changed'], values='prediction')
df_wide

analysis                                                                                                                            missing-previously-found-symbols  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/.libhogweed.so.6.4.hmac                      second/.libgmp.so.10.4.1.hmac                                               Unknown   
                              first/ANSI_X3.110.so                               second/ANSI_X3.110.so                                                          True   
                              first/B.so                                         second/B.so                                                                    True   
                              first/Base64.so                                    second/Base64.so                                                               True   
                              first/Byte.so                                      second/Byte.so                                                                 True   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/wait.so                                      second/wait.so                                                                 True   
                              first/xxlimited.cpython-310-x86_64-linux-gnu.so    second/xxlimited.cpython-310-x86_64-linux-gnu.so                               True   
                              first/xxlimited_35.cpython-310-x86_64-linux-gnu.so second/xxlimited_35.cpython-310-x86_64-linux-gn...                             True   
                              first/zink_dri.so                                  second/zink_dri.so                                                             True   
                              first/zlib.cpython-310-x86_64-linux-gnu.so         second/zlib.cpython-310-x86_64-linux-gnu.so                                    True   

analysis                                                                                                                            missing-previously-found-exports  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/.libhogweed.so.6.4.hmac                      second/.libgmp.so.10.4.1.hmac                                               Unknown   
                              first/ANSI_X3.110.so                               second/ANSI_X3.110.so                                                          True   
                              first/B.so                                         second/B.so                                                                    True   
                              first/Base64.so                                    second/Base64.so                                                               True   
                              first/Byte.so                                      second/Byte.so                                                                 True   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/wait.so                                      second/wait.so                                        

In [17]:
# Join analysis/predictor columns
df_wide = df_wide.reset_index()
df_wide.columns = ["_".join(names) for names in df_wide.columns]
df_wide.columns

Index(['a_', 'b_', 'original_', 'changed_',
       'missing-previously-found-symbols_symbols',
       'missing-previously-found-exports_symbols', 'abidiff_libabigail',
       'abi-compliance-tester_abi-laboratory'],
      dtype='object')

In [18]:
# Get predictions only
df_predictions = df_wide[['missing-previously-found-symbols_symbols',
                    'missing-previously-found-exports_symbols',
                    'abidiff_libabigail',
                    'abi-compliance-tester_abi-laboratory']]

In [19]:
# Filter to where predictions are either "True" or "False" only.
num_predictors = 4
df_predictions = df_predictions[(df_predictions == 'True').sum(axis=1) + (df_predictions == 'False').sum(axis=1) == num_predictors]

In [20]:
# Correlations between predictors.
(df_predictions == 'True').corr()

,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
missing-previously-found-symbols_symbols,NaN,NaN,NaN,NaN
missing-previously-found-exports_symbols,NaN,NaN,NaN,NaN
abidiff_libabigail,NaN,NaN,NaN,NaN
abi-compliance-tester_abi-laboratory,NaN,NaN,NaN,NaN
